In [2]:
# Selenium webdriver
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
# Waited conditionals for Selenium
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Dropdown selection support for selenium
from selenium.webdriver.support.select import Select
# Data Manipulations libraries
import pandas as pd
# Regular expressions library
import re
# Datetime functionalities
import time
# String manipulation
import string
# Mathematical operations
import numpy as np
# Biblioteca para leer archivos
import os
# Biblioteca para leer .env
from dotenv import load_dotenv
# Importamos la biblioteca para MongoDB
import pymongo
from pymongo import MongoClient
# Importar biblioteca para MySQL
from sqlalchemy import create_engine

## Usefull functions for the project

In [3]:
def initializeWebDriver():
    """
    This function enable to use the user browser to run automatizated 
    scripts to interact with it 
    """
    # Open the web browser
    driver = webdriver.Edge()
    # Maximize the window
    driver.maximize_window()
    
    return(driver)

In [4]:
def loginSIGMAA(driver, id_user, password):
    """
    Allows to login using user credentials into
    the SIGMAA
    """
    # Acceder al SIGMAA
    url = 'https://uclb.ucaribe.edu.mx/sigmaav2/'
    driver.get(url)

    # Buscar el campo de usuario y escribir el username
    userinput = driver.find_element(By.XPATH, '/html/body/div[2]/form/div/span[2]/input')
    userinput.send_keys(id_user)
    # Buscar el campo de contraseña y escribir el password
    passinput = driver.find_element(By.XPATH, '/html/body/div[2]/form/div/input')
    passinput.send_keys(password)
    # Buscar el botón de submit y dar clic para iniciar sesión
    submitinput = driver.find_element(By.XPATH, '/html/body/div[2]/form/button')
    submitinput.click()

In [22]:
def loginSIPP(driver, id_user, password):
    """
    Allows to login using user credentials into 
    the SIPP
    """

    ## Acceder al SIPP
    url = 'https://uclb.ucaribe.edu.mx/practicas/'
    driver.get(url)

    # Buscar el campo de usuario y escribir el username
    userinput = driver.find_element(By.XPATH, '/html/body/table/tbody/tr/td/form/table/tbody/tr[1]/td[2]/table/tbody/tr[1]/td/table/tbody/tr[1]/td[2]/input')
    userinput.send_keys(id_user)

    # Buscar el campo de contraseña y escribir el password
    passinput = driver.find_element(By.XPATH, '/html/body/table/tbody/tr/td/form/table/tbody/tr[1]/td[2]/table/tbody/tr[1]/td/table/tbody/tr[2]/td[2]/input')
    passinput.send_keys(password)
    # Buscar el botón de submit y dar clic para iniciar sesión
    submitinput = driver.find_element(By.XPATH, '/html/body/table/tbody/tr/td/form/table/tbody/tr[1]/td[2]/table/tbody/tr[1]/td/input')
    submitinput.click()

In [5]:
def getData(df, row, col):
    """
    This function allow us to fetch data depending on 
    a col and row number from a python dataframe
    """
    return(df.iloc[row, col])

In [6]:
def replaceNull(dictionary):
    """
    This function allows to replace nan values
    to False values in a recursive way
    """
    for k, v in dictionary.items():
        if type(v) == dict:
            replaceNull(v)

        else:
            if pd.isnull(v):
                dictionary[k] = False

    return(dictionary)

In [7]:
def getPersonalInfo(driver, id_user, password):
    """
    This function allow us to fetch all student 
    data from the personal information section on SIGMAA
    """
    personal_information_section = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div/div/div/ul[2]/li[6]/a')
    personal_information_section.click()

    # Extraer la tabla usando webscrapping
    dfs = pd.read_html(driver.page_source)
    df = dfs[1]
    
    # Obtener el nombre
    username = getData(df, 0, 5)
    username = string.capwords(username)
    
    # Obtener el primer apellido
    first_lastname = getData(df, 0, 1)
    first_lastname = string.capwords(first_lastname)

    # Obtener el segundo apellido
    second_lastname = getData(df, 0, 3)
    second_lastname = string.capwords(second_lastname)

    # Obtener la CURP
    curp = getData(df, 1, 3)

    # Obtener el RFC
    rfc = getData(df, 1, 1)

    # Obtener la nacionalidad
    nationality = getData(df, 2, 3)

    # Obtener numero de seguridad social
    nss = getData(df, 3, 1)

    # Obtener correo personal
    personal_email = getData(df, 3, 3)

    # Obtener la fecha de nacimiento
    birthday = getData(df, 2, 5)

    # Obtener el sexo
    sex = getData(df, 1, 5)

    # Obtener el telefono personal
    personal_phone = getData(df, 5, 3)

    # Obtener el telefono de casa
    home_phone = getData(df, 5, 1)

    # Obtener el estado civil
    dropdown = driver.find_element(By.XPATH, "/html/body/center/table/tbody/tr[4]/td[1]/select")
    dropdown = Select(dropdown)
    marital_status = dropdown.first_selected_option.text

    # Obtener el domicilio personal
    personal_address = getData(df, 4, 1)

    # Obtener el pais de nacimiento
    dropdown = driver.find_element(By.XPATH, "/html/body/center/table/tbody/tr[12]/td/select")
    dropdown = Select(dropdown)
    birthplace_country = dropdown.first_selected_option.text

    # Obtener el codigo de pais de nacimiento
    birthplace_country_code = driver.find_element(By.XPATH, "/html/body/center/table/tbody/tr[12]/td/select")
    birthplace_country_code = birthplace_country_code.get_attribute("value")

    # Obtener la entidad federativa de nacimiento
    dropdown = driver.find_element(By.XPATH, "/html/body/center/table/tbody/tr[13]/td/span/select")
    dropdown = Select(dropdown)
    birthplace_state = dropdown.first_selected_option.text

    # Obtener la ciudad de nacimiento
    dropdown = driver.find_element(By.XPATH, "/html/body/center/table/tbody/tr[14]/td/span/select")
    dropdown = Select(dropdown)
    birthplace_city = dropdown.first_selected_option.text

    # Obtener el nombre completo del padre
    father_fullname = getData(df, 8, 1)

    # Obtener el nombre completo de la madre
    mother_fullname = getData(df, 8, 3)

    # Obtener el estado civil de los padres
    dropdown = driver.find_element(By.XPATH, "/html/body/center/table/tbody/tr[10]/th[4]/select")
    dropdown = Select(dropdown)
    parents_marital_status = dropdown.first_selected_option.text

    # Obtener el pais del bachillerato de procedencia
    dropdown = driver.find_element(By.XPATH, "/html/body/center/table/tbody/tr[16]/td/select")
    dropdown = Select(dropdown)
    highschool_country = dropdown.first_selected_option.text

    # Obtener la entidad federativa del bachillerato de procedencia
    dropdown = driver.find_element(By.XPATH, "/html/body/center/table/tbody/tr[17]/td/span/select")
    dropdown = Select(dropdown)
    highschool_state = dropdown.first_selected_option.text

    # Obtener el municipio/condado del bachillerato de procedencia
    highschool_municipality_county = getData(df, 16, 1)

    # Obtener la ciudad del bachillerato de procedencia
    highschool_city = getData(df, 16, 4)

    # Obtener el nombre del bachillerato de procedencia
    highschool_school = getData(df, 17, 1)

    # Obtener el campus del bachillerato de procedencia
    highschool_campus = getData(df, 18, 1)

    # Obtener el sistema escolar del bachillerato de procedencia (publico o privado)
    school_system = getData(df, 18, 3)

    # Averiguar si el estudiante trabaja
    working = getData(df, 6, 1)
    if working == "NO":
        working = False
    else:
        working = True

    # Obtener el nombre de la empresa donde se labora
    company_name = getData(df, 6, 3)

    # Obtener la direccion de la empresa donde se labora
    company_address = getData(df, 6, 5)

    # Obtener el numero de telefono de la empresa donde se labora
    work_phone = getData(df, 5, 5)


    # Ir a la seccion de la boleta escolar en SIGMAA
    grades_section =  driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div/div/div/ul[2]/li[4]/a')
    grades_section.click()

    # Extraer la tabla usando webscrapping
    dfs = pd.read_html(driver.page_source)

    # Nos quedamos con la tabla de los datos generales de estudiante
    df = dfs[0]

    # Obtenemos el nombre de la carrera
    career_name = getData(df, 1, 1).split(" / ")[1]

    # Obtenemos la situacion del estudiante
    situation = getData(df, 2, 1)

    # Obtener el estatus del estudiante
    status = getData(df, 3, 1)

    # Averiguar si el estudiante ha liberado el servicio social o no
    social_service = getData(df, 5, 1)
    if social_service == "---":
        social_service = False
    else:
        social_service = True

    # Obtenemos el plan de estudios al que pertenece el estudiante
    study_plan = getData(df, 1, 1).split(" / ")[0]

    # (Pendiente) Datos que faltan obtener - Default por el momento
    department = "Ciencias Básicas e Ingenierías"
    profile_picture = "profiles/170300075.jpg"
    current_practices = "PID0201"

    # Creamos un diccionario para almancenar los datos
    # que hemos obtenido usando webscrapping
    personal_information = {
        "id_user" : id_user,
        "password" : password,
        "username" : username,
        "first_lastname" : first_lastname,
        "second_lastname" : second_lastname,
        "profile_picture" : profile_picture,
        "curp" : curp,
        "rfc" : rfc,
        "nationality" : nationality,
        "nss" : nss,
        "personal_email" : personal_email,
        "birthday" : birthday,
        "sex" : sex,
        "personal_phone" : personal_phone,
        "home_phone" : home_phone,
        "marital_status" : marital_status,
        "personal_address" : personal_address,
        "birthplace" : {
            "country" : birthplace_country,
            "country_code" : birthplace_country_code,
            "state": birthplace_state,
            "city" : birthplace_city
        },
        "parents" : {
            "father_fullname" : father_fullname,
            "mother_fullname" : mother_fullname,
            "parents_marital_status" : parents_marital_status 
        },
        "highschool" : {
            "country" : highschool_country,
            "state" : highschool_state,
            "municipality/county" : highschool_municipality_county,
            "city" : highschool_city,
            "school" : highschool_school,
            "campus" : highschool_campus,
            "school_system" : school_system
        },
        "career" : {
            "name" : career_name, 
            "situation" : situation, 
            "status" : status, 
            "social_service" : social_service,
            "current_practices"  : current_practices,
            "study_plan" : study_plan, 
            "department" : department
        },
        "job" : {
            "working": working, 
            "name" : company_name, 
            "address" : company_address, 
            "phone" : work_phone
        }
    }

    # Aplicamos una funcion que permite eliminar valores nulos
    # en el diccionario y reemplazarlos por un valor False
    # La funcion recorre el diccionario usando recursividad
    personal_information = replaceNull(personal_information)

    # Retornamos el diccionario con la informacion personal
    return(personal_information)

In [8]:
def logoutSIGMAA(driver):
    """
    Allows to logout from SIGMAA
    """
    # Encontrar el botón de cierre de sesión
    logout = driver.find_element(By.XPATH, '/html/body/div[2]/div/form/a')
    # Clic para cerrar sesión
    logout.click()

In [23]:
def logoutSIPP(driver):
    """
    Allows to logout from SIPP
    """
    # Encontrar el botón de cierre de sesión
    logout = driver.find_element(By.XPATH, '/html/body/div[3]/div/div/div[1]/ul/li[3]/a')
    # Clic para cerrar sesión
    logout.click()

In [9]:
def openMySQL(uri):
    """
    Creates a connection to database
    """
    # Creamos un engine usando la cadena de conexion y habilitando la conexion
    # segura a traves de Secure Socket Layer (SSL)
    engine = create_engine(uri + "?ssl=true")
    # Retornamos objeto engine
    return(engine)

In [10]:
def closeMySQL(engine):
    """
    Closes a MySQL connection
    """
    # Cerramos la sesión del engine
    engine.dispose()

In [11]:
def openMongoDB(uri):
    """
    Allows to open a mongoDB connection
    """
    # Creamon una conexion como cliente  
    # donde se recibe la cadena de conexión
    client = MongoClient(uri)
    # Retornamos el objeto cliente
    return(client)

In [24]:
def closeMongoDB(client):
    """
    Allows to close a mongoDB connection
    """
    # Cerramos el cliente
    client.close()

In [27]:
import inspect
print(inspect.getdoc(closeMongoDB))

Allows to close a mongoDB connection


In [13]:
def loadEnvs(path):
    """
    This function returns all the environmental variables
    used to stablish database connections in both MySQL and MongoDB
    """
    load_dotenv(path)

    # Obtenemos las variables las credenciales del usuario
    # (Development Only)
    id_user = os.getenv('ID')
    password = os.getenv('PASS')

    # Obtenemos las cadenas de conexion para MySQL y MongoDB
    MYSQL_ADDON_URI = os.getenv('MYSQL_ADDON_URI')
    MONGODB_URI = os.getenv('MONGODB_URI')
    
    return({"id_user": id_user, 
            "password" : password, 
            "MYSQL_ADDON_URI" : MYSQL_ADDON_URI, 
            "MONGODB_URI" : MONGODB_URI})

In [14]:
def getAcademicOffer(driver, study_plan):
    # Ir a la sección de las tablas de la oferta académica
    academic_offer_section = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div/div/div/ul[2]/li[1]/a')
    academic_offer_section.click()

    # xpath de pestañas
    tabs = [
        # Secciones
        '/html/body/center/ul/li[1]/a',
        # Talleres
        '/html/body/center/ul/li[2]/a',
        # Lengua Extranjera
        '/html/body/center/ul/li[3]/a'
    ]

    dataframes = dict()

    # Recorremos las pestañas y las enumeramos
    for index, t in enumerate(tabs):
        # Cambiamos a cada pestaña
        tab = driver.find_element(By.XPATH, t)
        tab.click() 

        # Extraer las tablas de la pestaña actual
        dfs = pd.read_html(driver.page_source)

        # Si el indice de la pestaña es 0
        if index == 0:
            # Nombres para las columnas
            columns = ['Tipo', 'Clave', 'Seccion', 'Asignatura', 'Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes', 'Sabado', 'U1', 'U2', 'U3', 'U4']
        else:
            # Nombres para las columnas
            columns = ['Clave', 'Seccion', 'Asignatura', 'Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes', 'Sabado', 'U1', 'U2', 'U3', 'U4', 'U5']

        # Columnas que se desean conservar
        desired_columns = [ 'Clave', 'Seccion', 'Asignatura', 'Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes', 'Sabado']

        # Recorre cada tabla en la pestaña actual
        for i in range(1, len(dfs)):
            # Renombra las columnas
            dfs[i] = dfs[i].set_axis(columns, axis=1)
            # Mantiene las columnas deseadas
            dfs[i] = dfs[i].loc[:, dfs[i].columns.isin(desired_columns)]

        # Para cada tabla en la pestaña actual
        for j in range(1, len(dfs)):
            # guardamos la tabla actual en una variable
            mydf = dfs[j]

            # Lista para guardar los tres campos a separa 
            # de la columna asignatura
            splits = []
            # Extraemos la lista de las asignaturas
            my_string = mydf['Asignatura']

            # Para cada asignatura
            for i in range(len(my_string)):
                # Dividimos la asignatura un espacio
                my_list = my_string[i].split('  ')
                # Los tres campos regresados se almacennan en una lista
                splits.append(my_list)

            # Tenemos tres listas nuevas
            # Para asignaturas
            subjects = []
            # Para profesores
            teachers = []
            # Para modalidades
            modalities = []

            # Para cada asignatura procesada
            # asignamos un dato a la lista correspondiente
            for i in splits:
                subjects.append(i[0])
                teachers.append(i[1])
                modalities.append(i[2])

            # Sobreescribimos la columna de asignaturas
            mydf['Asignatura'] = subjects

            # Insertamos a los profesores al dataframe
            mydf.insert(3, 'Profesor', teachers)
            # Insertamos las modalidades al dataframe
            mydf.insert(4, 'Modalidad', modalities)

            table_name = ""

            # Nombramos las tablas de acuerdo a su indice en las pestañas
            if index == 0:
                table_name = 'secciones_' + study_plan.lower()
            elif index == 1:
                table_name = 'talleres_' + study_plan.lower()
            else:
                table_name = 'lengua_extranjera_' + study_plan.lower()

            if j == 1:
                dataframes[table_name] = mydf
            else:
                dataframes[table_name].append(mydf)
    return(dataframes)

In [20]:
def getGrades(driver):
    """
    This function allows to obtain the average grades tables from SIGMAA
    and returns a dictionary where the key is the period and the value is
    the dataframe with the data
    """
    # Ir a la sección de la boleta escolar
    school_grades_section = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div/div/div/ul[2]/li[4]/a')
    school_grades_section.click()


    # Selecionar el munu desplehable del periodo de escolar
    select = driver.find_element(By.XPATH, "/html/body/center/form/select")
    select = Select(select)

    # Obtener todos los posibles valores de menu desplegable
    periods = [[option.get_attribute("value"), option.text] for option in select.options]

    grades = dict()

    for period, text in periods:
        if period != "":
            print("=======================================")
            print("Periodo escolar: ", text, end = "\n\n")
            # Selecionar el munu desplehable del periodo de escolar
            dropdown = driver.find_element(By.XPATH, "/html/body/center/form/select")
            dropdown.send_keys(text)

            # Consultar ese periodo
            button = driver.find_element(By.XPATH, '/html/body/center/form/input[4]')
            button.click()

            # Obtener la tabla de calificaciones
            dfs = pd.read_html(driver.page_source)
            school_grades = dfs[1]

            # Cambiar nombre de columnas
            columns = ['no.', 'type', 'section', 'subject', 'first_partial', 'second_partial', 'third_partial', 'average', 'final_grade', 'U1', 'U2']

            # Limpieza del dataframe de boleta
            # Cambiar nombre de columnas
            school_grades = school_grades.set_axis(columns, axis = 1)
            # Eliminar columnas no deseadas
            school_grades = school_grades.drop(['U1', 'U2'], axis = 1)
            # Eliminar filas con valores nulos
            school_grades.dropna(how = 'all', inplace = True)

            print("Nombre de columnas")
            print(list(school_grades.columns), end = "\n\n")

            print("Dataframe de boleta")
            print(school_grades, end = "\n\n")

            # Lista para guardar los tres campos a separa 
            # de la columna asignatura
            splits = []
            # Extraemos la lista de las asignaturas
            my_string = school_grades["subject"]

            # Para cada asignatura
            for i in range(len(my_string)):
                print("i: ", i)
                # Dividimos la asignatura un espacio
                my_list = my_string[i].split("  ")
                print("my_list = my_string[i].split('  '): ", my_list, end = "\n\n")

                # Separar la clave de asignatura
                my_list.insert(1, re.match(r"[A-Z]{3}[0-9]{4}|[A-Z]{2}[0-9]{4}", my_list[0]).group(0))
                print('my_list.insert(1, re.match(r"[A-Z]{2}[0-9]{4}", my_list[0]).group(0): ', my_list, end = "\n\n")

                # Separar el nombre de la asignatura
                my_list[0] = re.split(r"[A-Z]{3}[0-9]{4}|[A-Z]{2}[0-9]{4} ", my_list[0])[1]
                splits.append(my_list)
                print('my_list[0] = re.split(r"[A-Z]{3}[0-9]{4}|[A-Z]{2}[0-9]{4} ", my_list[0])[1]', my_list, end = "\n\n")

            # Tenemos tres listas nuevas
            # Para las claves de las asignaturas
            id_subjects = []
            # Para asignaturas
            subjects = []
            # Para profesores
            teachers = []
            # Para modalidades
            modalities = []

            # Para cada asignatura procesada
            # asignamos un dato a la lista correspondiente
            for i in splits:
                print("i splits :", i)
                subjects.append(i[0])
                id_subjects.append(i[1])
                teachers.append(i[2])
                modalities.append(i[3])

            print("subjects: ", subjects)
            print("id_subjects: ", id_subjects)
            print("teachers: ", teachers)
            print("modalities: ", modalities, end = "\n\n")

            # Sobreescribimos la columna de asignaturas
            school_grades["subject"] = subjects
            school_grades.insert(3, "id_subject", id_subjects)
            school_grades.insert(4, "teacher", teachers)
            school_grades.insert(5, "modality", modalities)

            print("Nuevas columnas")
            print(school_grades.columns)

            print("Dataframe de boleta actualizado")
            print(school_grades)
            
            # Guardamos la tabla de calificaciones actual en un diccionario
            # donde la llave es el periodo y el valor es el dataframe
            grades[period] = school_grades
    
    # Retornamos el diccionario con las calificaciones de todos
    # los periodos
    return(grades)

In [16]:
def getAverageGrade(grades_table):
    """
    This function allows to obtain the final average grade where the value is quantifiable
    and where values are not nulls (it occurs when there is no a grade available in the table)
    """
    # Obtener las calificaciones finales
    # de las asignaturas cuantificables
    grades_list = [float(g) for g in grades_table["final_grade"] if g not in ["Aprobado", "Reprobado"]]

    # Lista final de calificaciones sin valores nullos
    final_grades = []
    # Para cada calificacion cuantificable
    for gl in grades_list:
        # si no es un valor nulo, se adjunta a la lista
        if not np.isnan(gl):
            final_grades.append(gl)

    # print(final_grades)
    
    # Si la cantidad de las calificaciones es mayor a 0
    if len(final_grades) > 0:
        # Se calcula el promedio
        average_grades = np.mean(final_grades)
    # Si no hay calificaciones disponibles
    else:
        # El promedio por defecto es 0
        average_grades = 0

    # Retornamos el promedio de las calificaciones finales
    # del periodo escolar
    return(average_grades)

In [ ]:
def getPracticesOffer(driver):
    # Dar clic en el dropdown del menu
    dropdown = driver.find_element(By.XPATH, '/html/body/div[3]/div/div/div[1]/ul/li[2]/a')
    dropdown.click()

    # Ir a la oferta de proyectos
    projects_section = driver.find_element(By.XPATH, '/html/body/div[3]/div/div/div[1]/ul/li[2]/ul/li[2]/a')
    projects_section.click()

    # Obtener el selector
    select = driver.find_element(By.XPATH, "/html/body/div[3]/div/div/div[2]/div/div/div/div/div/table/tbody/tr/td[1]/table[1]/tbody/tr/td/form/fieldset/div/select")
    # Crear un objecto Select para interactuar con el
    select = Select(select)

    # Creamos una variable que almacene la lista de opciones
    # del dropdown
    options = []

    # Obtenemos una lista de las opciones disponibles en el dropdown
    for item in select.options:
        options.append(item.get_attribute("innerHTML"))

    # Print available dropdown options
    print("Opciones: ", options)

    for option in options:
        # Obtener el selector
        select = driver.find_element(By.XPATH, "/html/body/div[3]/div/div/div[2]/div/div/div/div/div/table/tbody/tr/td[1]/table[1]/tbody/tr/td/form/fieldset/div/select")
        # Consultamos la opcion de la lista desplegable
        select.send_keys(option)

        # Buscamos el boton para realizar la consulta
        button = driver.find_element(By.XPATH, "/html/body/div[3]/div/div/div[2]/div/div/div/div/div/table/tbody/tr/td[1]/table[1]/tbody/tr/td/form/fieldset/input")
        # Damos clic al boton para realizar la consulta
        button.click()

        # Obtenemos todas las tablas del periodo que consultamos
        dfs = pd.read_html(driver.page_source)

        # La ultima tabla es el dataframe con los datos de la oferta de practicas
        practices_offer = dfs[3]

        # Creamos una lista con los nuevos nombres para la tabla
        columns = ["No.", "Área de desempeño", "Ubicación", "Empresa", "Asignatura", "Espacios", "Opciones"]

        # Creamos una lista con los nombres de las columnas deseadas
        desired_columns = ["No.", "Área de desempeño", "Ubicación", "Empresa", "Espacios"]

        # Rename the columns
        practices_offer = practices_offer.set_axis(columns, axis = 1)

        # Filtramos las columnas con las que nos queremos quedar
        practices_offer = practices_offer.loc[:, practices_offer.columns.isin(desired_columns)]

        

In [17]:
def dataframe2Dict(dataframe):
    """
    This function allows to convert a dataframe to a
    dictionary (JSON like) object
    """
    # Convertir una dataframe a un diccionario
    data_dict = dataframe.to_dict("records")
    # Retornamos el diccionario
    return(data_dict)

## Get the student info when registered for first time

In [21]:
# Load environmental variables
env = loadEnvs("../../.env")

# Start the webdriver
driver = initializeWebDriver()

# Login into SIGMAA using the student credentials
loginSIGMAA(driver, 
            env["id_user"], 
            env["password"])

##########################################################
#              Get personal information                  #
##########################################################

# Get all the student information
student_information = getPersonalInfo(driver, 
                                      env["id_user"], 
                                      env["password"])

# Open a MongoDB connection
client = openMongoDB(env["MONGODB_URI"])

# Use a database and select a collection
db = client["sure"]
collection = db["users"]

# Save student information to MongoDB database
collection.insert_one(student_information)

# Close MongoDB connection
closeMongoDB(client)

##########################################################
#              Get academic offer                        #
##########################################################

# Get the academic offer
academic_offer = getAcademicOffer(driver, "2019IA")

# Open MySQL connection
engine = openMySQL(env["MYSQL_ADDON_URI"])

# Save the academic offer to mysql database
for table_name, df in academic_offer.items():
    df.to_sql(table_name, con = engine, if_exists = "replace", index = False)

# Close MySQL connection
closeMySQL(engine)

##########################################################
#              Get practices offer                       #
##########################################################


##########################################################
#              Get social service offer                  #
##########################################################


##########################################################
#              Get payments                              #
##########################################################


##########################################################
#              Get grades                                #
##########################################################

# Get all the grades tables
grades = getGrades(driver)

student_grades = {}
student_grades["id_user"] = env["id_user"]
student_grades["grades"] = {}

for period, dataframe in grades.items():
    student_grades["grades"].update({
        period : {
            "average_grades" : getAverageGrade(dataframe),
            "total_credits" : None,
            "subject_grades" : dataframe2Dict(dataframe)
        }
    })

# Open a MongoDB connection
client = openMongoDB(env["MONGODB_URI"])

# Use a database and select a collection
db = client["sure"]
collection = db["grades"]

# Save student information to MongoDB database
collection.insert_one(student_grades)

# Close MongoDB connection
closeMongoDB(client)

# Close the SIGMAA session
logoutSIGMAA(driver)

# Close the webdriver
driver.quit()

C:\Users\kenne\AppData\Local\Temp\ipykernel_9892\220568674.py:99: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframes[table_name].append(mydf)
C:\Users\kenne\AppData\Local\Temp\ipykernel_9892\220568674.py:99: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframes[table_name].append(mydf)
C:\Users\kenne\AppData\Local\Temp\ipykernel_9892\220568674.py:99: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframes[table_name].append(mydf)


Periodo escolar:  202201 Primavera

Nombre de columnas
['no.', 'type', 'section', 'subject', 'first_partial', 'second_partial', 'third_partial', 'average', 'final_grade']

Dataframe de boleta
   no. type  section                                            subject  \
0  1.0  OP1      1.0  II0106 Cálculo diferencial  Zavala Amezcua / E...   
1  2.0  OP1      1.0  IA0105 Química Inorgánica García Rosas / Marin...   
2  3.0  OP1      1.0  II0104 Medio ambiente y desarrollo sustentable...   
3  4.0  OP1      1.0  IA0160 Economía ambiental  Muñoz Dzib / Melcho...   
4  5.0  OP1      1.0  LI1103 Nivel 3 Inglés  Rodríguez Dacak / Gabri...   
5  6.0  OP1      1.0  DP0001 Propedéutico de habilidades básicas de ...   
6  7.0  OP1      1.0  DP0193 Taller de redacción profesional  Caamal...   
7  8.0  OP1      2.0  II0102 Física experimental  Ramírez Carmona / ...   

  first_partial second_partial third_partial     average final_grade  
0           9.0            7.0           7.0  7.66666651     

IndexError: list index out of range

## Pruebas

In [4]:
env["id_user"]

'210300580'

In [ ]:
# Load environmental variables
env = loadEnvs("../../.env")

# Start the webdriver
driver = initializeWebDriver()

# Login into SIGMAA using the student credentials
loginSIGMAA(driver, 
            env["id_user"], 
            env["password"])

In [ ]:
dataframe2Dict(grades["202201"])

In [ ]:
student_grades = {}
student_grades["id_user"] = env["id_user"]
student_grades

In [ ]:
for period, dataframe in grades.items():
    student_grades.update({"grades" : {
        "id_period" : period,
        "average_grades" : getAverageGrade(dataframe),
        "total_credits" : np.nan,
        "subject_grades" : dataframe2Dict(dataframe)
    }})

In [ ]:
student_grades

In [ ]:
# Load environmental variables
env = loadEnvs("../../.env")

## To Database

In [ ]:
index = 0

# Open MySQL connection
engine = openMySQL(env["MYSQL_ADDON_URI"])

for table_name, df in grades.items():
    # Save the academic offer to mysql database
    for table_name, df in grades.items():
        if index == 0:
            df.to_sql(table_name+"_"+env["id_user"], con = engine, if_exists = "replace", index = False)
        else:
            df.to_sql(table_name+"_"+env["id_user"], con = engine, if_exists = "append", index = False)
        index = index + 1

# Close MySQL connection
closeMySQL(engine)

In [ ]:
# Close the SIGMAA session
logoutSIGMAA(driver)

# Close the webdriver
driver.quit()